In [49]:
# Useful Libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
import sklearn
import random
import time 

#Data Modelling Libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import IPython
from IPython import display #for prettier DataFrames

#Visualization
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

# Input data files are available in the "../input/" directory.
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


**I. Data Exploration**

**Problem definition:** We want to predict the success of a movie based on various features. This means we want to be able to compute the revenue a movie will generate when given its characteristics. 

In [50]:
data_raw = pd.read_csv('../input/train.csv')
#We break our dataset into 3 splits : train, test, and validation. 
    #The test file provided is the validation file that we will use for submission.
data_val = pd.read_csv('../input/test.csv')
    #We copy the train set so that we can make changes without worrying about it. 
        #Reminder if I want to implement data_raw.copy(deep = True) : Deep copy creates new id's of every object it contains while 
        #normal copy only copies the elements from the parent and creates a new id for a variable to which it is copied to.
data1 = data_raw.copy(deep = True)
    #We pass data1 & data_val by reference to clean them both at once
data_cleaner = [data1, data_val]

In [51]:
#preview data : 
print ('Info on our train set\n')
print (data_raw.info())



Info on our train set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
id                       3000 non-null int64
belongs_to_collection    604 non-null object
budget                   3000 non-null int64
genres                   2993 non-null object
homepage                 946 non-null object
imdb_id                  3000 non-null object
original_language        3000 non-null object
original_title           3000 non-null object
overview                 2992 non-null object
popularity               3000 non-null float64
poster_path              2999 non-null object
production_companies     2844 non-null object
production_countries     2945 non-null object
release_date             3000 non-null object
runtime                  2998 non-null float64
spoken_languages         2980 non-null object
status                   3000 non-null object
tagline                  2403 non-null object
title                    3000 non-null objec

In [52]:
print ('What it looks like at the top\n')
data_raw.head()

What it looks like at the top



,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [53]:
print ('What it looks like at the bottom\n')
data_raw.tail()

What it looks like at the bottom



,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
2995,2996,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0109403,en,Chasers,Military men Rock Reilly and Eddie Devane are ...,9.853270,/j8Q7pQ27hvH54wpxJzIuQgQCdro.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",4/22/94,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It was supposed to be a routine prisoner trans...,Chasers,"[{'id': 378, 'name': 'prison'}, {'id': 572, 'n...","[{'cast_id': 2, 'character': 'Rock Reilly', 'c...","[{'credit_id': '52fe4494c3a368484e02ac7d', 'de...",1596687
2996,2997,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,tt2364975,sv,Vi är bäst!,Three girls in 1980s Stockholm decide to form ...,3.727996,/sS01LSy6KDrCZAhtkO18UdnWFT1.jpg,"[{'name': 'Memfis Film', 'id': 321}, {'name': ...","[{'iso_3166_1': 'DK', 'name': 'Denmark'}, {'is...",3/28/13,102.0,"[{'iso_639_1': 'sv', 'name': 'svenska'}]",Released,NaN,We Are the Best!,"[{'id': 1192, 'name': 'sweden'}, {'id': 4470, ...","[{'cast_id': 5, 'character': 'Bobo', 'credit_i...","[{'credit_id': '5716b72ac3a3686678012c84', 'de...",180590
2997,2998,NaN,65000000,"[{'id': 80, 'name': 'Crime'}, {'id': 28, 'name...",NaN,tt0116908,en,The Long Kiss Goodnight,"Samantha Caine, suburban homemaker, is the ide...",14.482345,/4MENR8x6mYqnZvp2hGjSaPJz64J.jpg,"[{'name': 'New Line Cinema', 'id': 12}, {'name...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/11/96,120.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,What's forgotten is not always gone.,The Long Kiss Goodnight,"[{'id': 441, 'name': 'assassination'}, {'id': ...","[{'cast_id': 10, 'character': 'Samantha Caine ...","[{'credit_id': '52fe443a9251416c7502d579', 'de...",89456761
2998,2999,NaN,42000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",http://www.alongcamepolly.com/,tt0343135,en,Along Came Polly,Reuben Feffer is a guy who's spent his entire ...,15.725542,/nIY4kvJTTnxoBR0wycrXng5MOYs.jpg,"[{'name': 'Jersey Films', 'id': 216}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",1/16/04,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"For the most cautious man on Earth, life is ab...",Along Came Polly,"[{'id': 966, 'name': 'beach'}, {'id': 2676, 'n...","[{'cast_id': 8, 'character': 'Reuben Feffer', ...","[{'credit_id': '556f817b9251410866000a63', 'de...",171963386
2999,3000,NaN,35000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",http://www.abductionthefilm.com/,tt1600195,en,Abduction,A young man sets out to uncover the truth abou...,10.512109,/cUT6NQP5LAJpmUoStGtXmvNt4zA.jpg,"[{'name': 'Lions Gate Films', 'id': 35}, {'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",9/22/11,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,They stole his life. He's taking it back.,Abduction,"[{'id': 591, 'name': 'cia'}, {'id': 822, 'name...","[{'cast_id': 2, 'character': 'Nathan Harper', ...","[{'credit_id': '5391990d0e0a260fb5001629', 'de...",82087155


In [54]:
data_raw.sample(10)
#There is going to be some cleaning to do in the way variables are coded

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
2306,2307,"[{'id': 210303, 'name': 'Rebuild of Evangelion...",0,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...",http://www.evangelion.co.jp/,tt0860906,ja,ヱヴァンゲリヲン新劇場版：破,Under constant attack by monstrous creatures c...,6.518028,/odCtbDVJMxaqsTt4fIkM7UHZ8ta.jpg,"[{'name': 'Studio Khara', 'id': 4145}]","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",6/26/09,108.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,"In The War Between Heaven and Earth, Salvation...",Evangelion: 2.0 You Can (Not) Advance,NaN,"[{'cast_id': 4, 'character': 'Shinji Ikari', '...","[{'credit_id': '52fe4452c3a368484e01c79b', 'de...",41162041
1140,1141,NaN,20000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0118799,it,La vita è bella,A touching story of an Italian book seller of ...,39.394970,/f7DImXDebOs148U4uPjI61iDvaK.jpg,"[{'name': 'Miramax Films', 'id': 14}, {'name':...","[{'iso_3166_1': 'IT', 'name': 'Italy'}]",12/20/97,116.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"An unforgettable fable that proves love, famil...",Life Is Beautiful,"[{'id': 131, 'name': 'italy'}, {'id': 483, 'na...","[{'cast_id': 7, 'character': 'Dora', 'credit_i...","[{'credit_id': '52fe4262c3a36847f801a10b', 'de...",229400000
1484,1485,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.thebookthief.com/,tt0816442,en,The Book Thief,While subjected to the horrors of WWII Germany...,16.373622,/rUJbwJkJNjwEcFYzEktOrgm5YUd.jpg,"[{'name': 'Studio Babelsberg', 'id': 264}, {'n...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",11/8/13,131.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Courage beyond words.,The Book Thief,"[{'id': 1956, 'name': 'world war ii'}, {'id': ...","[{'cast_id': 1, 'character': 'Hans Hubermann',...","[{'credit_id': '53185c8f92514177bf000e45', 'de...",76586316
765,766,NaN,105000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",https://www.facebook.com/PointBreakMovie,tt2058673,en,Point Break,A young undercover FBI agent infiltrates a gan...,7.291872,/4zfbf70df2ySChbsIEjhAC9oXJ1.jpg,"[{'name': 'Studio Babelsberg', 'id': 264}, {'n...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",12/3/15,114.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The only law that matters is gravity,Point Break,"[{'id': 1568, 'name': 'undercover'}, {'id': 46...","[{'cast_id': 4, 'character': 'Bodhi', 'credit_...","[{'credit_id': '530ccb429251411435001272', 'de...",133718711
1818,1819,"[{'id': 10, 'name': 'Star Wars Collection', 'p...",120000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,tt0121765,en,Star Wars: Episode II - Attack of the Clones,"Ten years after the invasion of Naboo, the gal...",14.072511,/2vcNFtrZXNwIcBgH5e2xXCmVR8t.jpg,"[{'name': 'Lucasfilm', 'id': 1}]","[{'iso_3166_1': 'US', 'name': 'United States o...",5/15/02,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Jedi Shall Not Know Anger. Nor Hatred. Nor L...,Star Wars: Episode II - Attack of the Clones,"[{'id': 1399, 'name': 'senate'}, {'id': 5340, ...","[{'cast_id': 7, 'character': 'Obi Wan Kenobi',...","[{'credit_id': '52fe431fc3a36847f803bd8b', 'de...",649398328
1932,1933,NaN,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...",http://disney.go.com/disneypictures/the-odd-li...,tt1462769,en,The Odd Life of Timothy Green,A childless couple bury a box in their backyar...,5.426921,/5eO9ajHlNgR6hp0UFO0d6b3PjNQ.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/15/12,105.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,He's a force of nature.,The Odd Life of Timothy Green,"[{'id': 6348, 'name': 'green'}, {'id': 188086,...","[{'cast_id': 3, 'character': 'Cindy Green', 'c...","[{'c

In [55]:
print ('Train columns with null values:\n', data1.isnull().sum())
print ('-' * 25)
print ('Test columns with null values:\n', data_val.isnull().sum())
print ('-' * 25)

Train columns with null values:
 id                          0
belongs_to_collection    2396
budget                      0
genres                      7
homepage                 2054
imdb_id                     0
original_language           0
original_title              0
overview                    8
popularity                  0
poster_path                 1
production_companies      156
production_countries       55
release_date                0
runtime                     2
spoken_languages           20
status                      0
tagline                   597
title                       0
Keywords                  276
cast                       13
crew                       16
revenue                     0
dtype: int64
-------------------------
Test columns with null values:
 id                          0
belongs_to_collection    3521
budget                      0
genres                     16
homepage                 2978
imdb_id                     0
original_language         

* "belongs to collection" has many null values, almost all of them actually. They're not missing values, only a few movies belong to a collection, e.g. Dracula. It will be interesting to see if this boosts revenues in any way. 
* "homepage" doesn't seem very pertinent. Most movies don't have a homepage. However, it is possible that the movies with a homepage are the one that have a big budget, hence are more likely to be successful.
* "production_companies" have some missing values. It might be the indie movies.
* "production_countries" have missing values as well, we might want to replace them with the most popular producing countries. 
* "tagline" : some movies don't have a tagline. It might be interesting to see if it has an effect on revenue. Side note : we could run an NLP algorithm on the taglines to see if a kind of tagline works better than another.
* "spoken language" is definitely a missing value we have to fill. Should we look online for that data, or just replace it by English? Or are they 42 mute movies?
* "keywords" is very valuable as well. We might be able to complete it with information from the tagline and the title.

In [56]:
data_raw.describe(include = 'all')

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
count,3000.000000,604,3.000000e+03,2993,946,3000,3000,3000,2992,3000.000000,2999,2844,2945,3000,2998.000000,2980,3000,2403,3000,2724,2987,2984,3.000000e+03
unique,NaN,422,NaN,872,941,3000,36,2975,2992,NaN,2999,2383,321,2398,NaN,401,2,2400,2969,2648,2975,2984,NaN
top,NaN,"[{'id': 645, 'name': 'James Bond Collection', ...",NaN,"[{'id': 18, 'name': 'Drama'}]",http://www.transformersmovie.com/,tt0436697,en,Alone in the Dark,"Cheated on, mistreated and stepped on, the wom...",NaN,/o4gaHLrCIC2rHNxdsfYX427ZVYI.jpg,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'US', 'name': 'United States o...",9/10/10,NaN,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Based on a true story.,Hot Pursuit,"[{'id': 10183, 'name': 'independent film'}]",[],"[{'credit_id': '52fe48fbc3a368484e114981', 'de...",NaN
freq,NaN,16,NaN,266,4,1,2575,2,1,NaN,1,51,1752,5,NaN,1817,2996,3,2,27,13,1,NaN
mean,1500.500000,NaN,2.253133e+07,NaN,NaN,NaN,NaN,NaN,NaN,8.463274,NaN,NaN,NaN,NaN,107.856571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.672585e+07
std,866.169729,NaN,3.702609e+07,NaN,NaN,NaN,NaN,NaN,NaN,12.104000,NaN,NaN,NaN,NaN,22.086434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.375323e+08
min,1.000000,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,0.000001,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
25%,750.750000,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,4.018053,NaN,NaN,NaN,NaN,94.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.379808e+06
50%,1500.500000,NaN,8.000000e+06,NaN,NaN,NaN,NaN,NaN,NaN,7.374861,NaN,NaN,NaN,NaN,104.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.680707e+07
75%,2250.250000,NaN,2.900000e+07,NaN,NaN,NaN,NaN,NaN,NaN,10.890983,NaN,NaN,NaN,NaN,118.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.891920e+07


**Variables we can delete : **
- imdb_id
- original_title
- poster_path : unless we do some image analysis
- status : aren't most of them "released"?

**>Replacing values**

In [89]:
# Clean the 'belongs_to_collection' column 
# --> apply() : Apply a function along an axis of the DataFrame
data1["collection_bin"] = data1["belongs_to_collection"].apply(lambda x: 1 if not pd.isnull(x) else np.nan)

col = 'belongs_to_collection'
data_collec = data1[['id', col]]

collections = {}

for _, x in data_collec.values:
    
    value = eval(str(x))
    print(value)


def collection_one_hot_encoder(collec : str):
    """ Function to encode movies with 1
         if they belong to the same collection
         and 0 if not.
    """
    data1[collec] = data1["belongs_to_collection"].apply(lambda x: 1 if collec in str(x) else 0)
    


[{'id': 313576, 'name': 'Hot Tub Time Machine Collection', 'poster_path': '/iEhb00TGPucF0b4joM1ieyY026U.jpg', 'backdrop_path': '/noeTVcgpBiD48fDjFVic1Vz7ope.jpg'}]
[{'id': 107674, 'name': 'The Princess Diaries Collection', 'poster_path': '/wt5AMbxPTS4Kfjx7Fgm149qPfZl.jpg', 'backdrop_path': '/zSEtYD77pKRJlUPx34BJgUG9v1c.jpg'}]


NameError: name 'nan' is not defined

In [83]:
value = eval(str(data_collec.values[1]))

SyntaxError: invalid syntax (<string>, line 2)